In [1]:
import os
import pandas as pd
import numpy as np

tables = {}
table_names = []
for f in os.listdir('./parquet'):
    table_names.append(f.split('.')[0])
    tables[f.split('.')[0]] = pd.read_parquet(f'./parquet/{f}')
table_keys = {i: tables[i].columns.values for i in tables}

In [2]:
tables['TEXT_CODES'].columns.values

array(['tts_sysno', 'c_textid', 'c_title_chn', 'c_suffix_version',
       'c_title', 'c_title_trans', 'c_text_type_id', 'c_text_year',
       'c_text_nh_code', 'c_text_nh_year', 'c_text_range_code',
       'c_period', 'c_bibl_cat_code', 'c_extant', 'c_text_country',
       'c_text_dy', 'c_pub_country', 'c_pub_dy', 'c_pub_year',
       'c_pub_nh_code', 'c_pub_nh_year', 'c_pub_range_code', 'c_pub_loc',
       'c_publisher', 'c_pub_notes', 'c_source', 'c_pages',
       'c_secondary_source_author', 'c_url_api', 'c_url_api_coda',
       'c_url_homepage', 'c_notes', 'c_number', 'c_counter',
       'c_title_alt_chn', 'c_created_by', 'c_created_date',
       'c_modified_by', 'c_modified_date'], dtype=object)

In [3]:
neighbor = {}
intermediate = []
for i in table_names:
    if len(table_keys[i]) == 2:
        intermediate.append(i)
    for j in table_names:
        if i==j:
            continue
        if table_keys[i][0] in table_keys[j]:
            if i not in neighbor:
                neighbor[i] = {j: None}
            else:
                neighbor[i].update({j: None})
print(len(neighbor.keys()))
neighbor

77


{'ENTRY_DATA': {'OFFICE_CODES': None,
  'BIOG_INST_DATA': None,
  'STATUS_DATA': None,
  'EVENTS_DATA': None,
  'ALTNAME_DATA': None,
  'POSTED_TO_ADDR_DATA': None,
  'TEXT_INSTANCE_DATA': None,
  'ASSOC_DATA': None,
  'BIOG_ADDR_DATA': None,
  'TEXT_CODES': None,
  'POSTING_DATA': None,
  'KIN_DATA': None,
  'BIOG_TEXT_DATA': None,
  'POSTED_TO_OFFICE_DATA': None,
  'BIOG_MAIN': None},
 'PLACE_CODES': {'ADDR_PLACE_DATA': None},
 'SOCIAL_INSTITUTION_NAME_CODES': {'ENTRY_DATA': None,
  'BIOG_INST_DATA': None,
  'ASSOC_DATA': None,
  'SOCIAL_INSTITUTION_CODES': None,
  'SOCIAL_INSTITUTION_ALTNAME_DATA': None,
  'POSTED_TO_OFFICE_DATA': None,
  'SOCIAL_INSTITUTION_ADDR': None,
  'SOCIAL_INSTITUTION_CODES_CONVERSION': None},
 'ENTRY_TYPES': {'ENTRY_CODE_TYPE_REL': None},
 'OFFICE_CODES': {'ENTRY_DATA': None,
  'BIOG_INST_DATA': None,
  'STATUS_DATA': None,
  'EVENTS_DATA': None,
  'ALTNAME_DATA': None,
  'POSTED_TO_ADDR_DATA': None,
  'TEXT_INSTANCE_DATA': None,
  'ASSOC_DATA': None,
  'BI

In [4]:
for i in neighbor:
    for j in neighbor[i]:
        if table_keys[i][0] == table_keys[j][0]:
            neighbor[i][j] = '一对一'
        else:
            neighbor[i][j] = '一对多'
            

In [5]:
reverse = {}
for i in neighbor:
    for j in neighbor[i]:
        if neighbor[i][j] == '一对多' and j in intermediate:
            if j not in reverse:
                reverse[j] = [i]
            else:
                reverse[j].append(i)

In [6]:
reverse

{'ENTRY_CODE_TYPE_REL': ['ENTRY_TYPES'],
 'CopyMissingTables': ['CopyTables', 'TablesFieldsChanges'],
 'CopyTablesDefault': ['CopyTables', 'TablesFieldsChanges'],
 'ASSOC_CODE_TYPE_REL': ['ASSOC_TYPES'],
 'STATUS_CODE_TYPE_REL': ['STATUS_TYPES'],
 'TEXT_BIBLCAT_CODE_TYPE_REL': ['TEXT_BIBLCAT_TYPES']}

In [10]:
df = pd.DataFrame(columns=['表一', '表二', '表间关系'])
for i in neighbor:
    for j in neighbor[i]:
        df.loc[len(df)] = [i, j, neighbor[i][j]]

In [12]:
for i in reverse:
    if len(reverse[i])==2:
        df.loc[len(df)] = [reverse[i][0], reverse[i][1], '多对多']

In [14]:
with open('./表间关系.csv', 'w', encoding='utf-8') as f:
    df.to_csv(f, index=False)